#Clase ternaria

In [2]:
import os, sys
print(os.getcwd())   # dónde está parado el notebook
print(sys.path)      # lista de rutas donde busca módulos
print(os.listdir())  # qué archivos hay en el directorio actual
import sys, os
import polars as pl


c:\Users\Flor\Documents\UBA\DMEyF
['C:\\Program Files\\WindowsApps\\PythonSoftwareFoundation.Python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0\\python311.zip', 'C:\\Program Files\\WindowsApps\\PythonSoftwareFoundation.Python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0\\DLLs', 'C:\\Program Files\\WindowsApps\\PythonSoftwareFoundation.Python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0\\Lib', 'C:\\Program Files\\WindowsApps\\PythonSoftwareFoundation.Python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0', 'c:\\Users\\Flor\\Documents\\UBA\\DMEyF\\venv', '', 'c:\\Users\\Flor\\Documents\\UBA\\DMEyF\\venv\\Lib\\site-packages', 'c:\\Users\\Flor\\Documents\\UBA\\DMEyF\\venv\\Lib\\site-packages\\win32', 'c:\\Users\\Flor\\Documents\\UBA\\DMEyF\\venv\\Lib\\site-packages\\win32\\lib', 'c:\\Users\\Flor\\Documents\\UBA\\DMEyF\\venv\\Lib\\site-packages\\Pythonwin']
['.git', '.gitignore', 'Clase_ternaria.py', 'compe_01.ipynb', 'data', 'FeatureEng_video.ipynb', 'logs', 'main.py', 'mi_proyecto.duckdb', 'modelo.ipynb', 'predicciones_kaggle

In [3]:
# Supongamos que el archivo se llama "datos.csv"
df = pl.read_csv("data/competencia_01_con_clase_ternaria.csv")


In [4]:
import polars as pl
# Mostrar todos los registros del cliente 930544172
cliente_id = 930544172

df_cliente = df.filter(pl.col("numero_de_cliente") == cliente_id)

print(df_cliente)

shape: (2, 155)
┌───────────┬──────────┬───────────┬───────────┬───┬───────────┬───────────┬───────────┬───────────┐
│ numero_de ┆ foto_mes ┆ active_qu ┆ cliente_v ┆ … ┆ Visa_ccon ┆ Visa_cade ┆ Visa_mpag ┆ clase_ter │
│ _cliente  ┆ ---      ┆ arter     ┆ ip        ┆   ┆ sumos     ┆ lantosefe ┆ ominimo   ┆ naria     │
│ ---       ┆ i64      ┆ ---       ┆ ---       ┆   ┆ ---       ┆ ctivo     ┆ ---       ┆ ---       │
│ i64       ┆          ┆ i64       ┆ i64       ┆   ┆ i64       ┆ ---       ┆ f64       ┆ str       │
│           ┆          ┆           ┆           ┆   ┆           ┆ i64       ┆           ┆           │
╞═══════════╪══════════╪═══════════╪═══════════╪═══╪═══════════╪═══════════╪═══════════╪═══════════╡
│ 930544172 ┆ 202101   ┆ 1         ┆ 0         ┆ … ┆ 11        ┆ 0         ┆ 1137.81   ┆ BAJA+2    │
│ 930544172 ┆ 202102   ┆ 1         ┆ 0         ┆ … ┆ 9         ┆ 0         ┆ 1794.69   ┆ BAJA+1    │
└───────────┴──────────┴───────────┴───────────┴───┴───────────┴───────────

#Feature Engeneering

In [5]:
type(df)
# debe devolver: <class 'polars.internals.frame.DataFrame'>


polars.dataframe.frame.DataFrame

In [6]:
#shape df
df.shape

(978439, 155)

In [7]:
# Supongamos que tu DataFrame se llama df
# Detectar automáticamente columnas numéricas
numeric_cols = [
    col for col, dtype in df.schema.items()
    if dtype in [pl.Int8, pl.Int16, pl.Int32, pl.Int64, pl.Float32, pl.Float64]
]

import duckdb
import polars as pl

def add_running_min_max_polars(df: pl.DataFrame) -> pl.DataFrame:
    """
    Calcula columnas _min y _max acumuladas por cliente hasta cada fila,
    usando DuckDB y devolviendo un Polars DataFrame.
    
    Mantiene todas las columnas originales y agrega <col>_min y <col>_max
    para las columnas numéricas.
    
    Parámetros:
    -----------
    df : pl.DataFrame
        DataFrame con columnas 'numero_de_cliente', 'foto_mes' y otras columnas numéricas y no numéricas.
        
    Retorna:
    --------
    pl.DataFrame con las columnas originales + <col>_min y <col>_max.
    """
    # columnas numéricas a procesar (excluyendo id y tiempo)
    numeric_cols = [c for c, dt in zip(df.columns, df.dtypes)
                    if dt in [pl.Int32, pl.Int64, pl.Float32, pl.Float64]
                    and c not in ["numero_de_cliente", "foto_mes"]]

    # registrar DataFrame en DuckDB
    con = duckdb.connect(database=":memory:")
    con.register("df", df)

    # construir query dinámica
    select_exprs = df.columns.copy()  # todas las columnas originales
    for col in numeric_cols:
        select_exprs.append(
            f"MIN({col}) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS {col}_min"
        )
        select_exprs.append(
            f"MAX({col}) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS {col}_max"
        )

    query = f"SELECT {', '.join(select_exprs)} FROM df"

    # ejecutar la query y obtener el resultado como Polars
    df_result = pl.from_arrow(con.execute(query).arrow())

    con.close()
    return df_result
# aplicar la función
df_final = add_running_min_max_polars(df)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [8]:
#shape df_final
# debe devolver: (136473, 122)
df_final.shape

(978439, 455)

In [9]:
import polars as pl
import duckdb

def add_diff_previous_months_to_df(df_final: pl.DataFrame, df_base: pl.DataFrame) -> pl.DataFrame:
    """
    Calcula la diferencia de cada columna numérica original entre el mes actual
    y el mes anterior, y entre el mes actual y dos meses atrás, para cada cliente.
    Agrega las columnas al df_final.
    
    Parámetros:
    -----------
    df_final : pl.DataFrame
        DataFrame que contiene ya las columnas originales + min/max acumuladas.
    df_base : pl.DataFrame
        DataFrame original con las columnas numéricas base y 'numero_de_cliente', 'foto_mes'.
        
    Retorna:
    --------
    df_final : pl.DataFrame
        El mismo df_final con las columnas <col>_diff_prev y <col>_diff_prev2 agregadas.
    """
    # columnas numéricas originales (sin min/max)
    numeric_cols = [c for c, dt in zip(df_base.columns, df_base.dtypes)
                    if dt in [pl.Int32, pl.Int64, pl.Float32, pl.Float64]
                    and c not in ["numero_de_cliente", "foto_mes"]]

    # registrar df_base en DuckDB
    con = duckdb.connect(database=":memory:")
    con.register("df", df_base)

    # construir query SQL para calcular diferencias
    select_exprs = ["numero_de_cliente", "foto_mes"]
    for col in numeric_cols:
        # diferencia con el mes anterior
        select_exprs.append(
            f"({col} - LAG({col}) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes)) AS {col}_diff_prev"
        )
        # diferencia con dos meses atrás
        select_exprs.append(
            f"({col} - LAG({col}, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes)) AS {col}_diff_prev2"
        )

    query = f"SELECT {', '.join(select_exprs)} FROM df"
    
    # ejecutar query y devolver resultados como Polars
    df_diff = pl.from_arrow(con.execute(query).arrow())
    con.close()

    # unir df_diff con df_final usando numero_de_cliente y foto_mes
    df_final = df_final.join(df_diff, on=["numero_de_cliente", "foto_mes"], how="left")

    return df_final

# df: DataFrame original con solo las columnas base
# df_final: DataFrame que ya tiene columnas _min y _max
df_final = add_diff_previous_months_to_df(df_final, df)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [10]:
df_final.shape

(978439, 755)

In [11]:
#tiene df_final la columna "clase_ternaria"?
print("clase_ternaria" in df_final.columns) 

True


In [12]:
# 7. Guardar el dataframe resultante en un nuevo CSV
df_final.write_csv("df_confe.csv")

In [13]:
import duckdb


In [14]:
df_final.head(
    
)

numero_de_cliente,foto_mes,active_quarter,cliente_vip,internet,cliente_edad,cliente_antiguedad,mrentabilidad,mrentabilidad_annual,mcomisiones,mactivos_margen,mpasivos_margen,cproductos,tcuentas,ccuenta_corriente,mcuenta_corriente_adicional,mcuenta_corriente,ccaja_ahorro,mcaja_ahorro,mcaja_ahorro_adicional,mcaja_ahorro_dolares,cdescubierto_preacordado,mcuentas_saldo,ctarjeta_debito,ctarjeta_debito_transacciones,mautoservicio,ctarjeta_visa,ctarjeta_visa_transacciones,mtarjeta_visa_consumo,ctarjeta_master,ctarjeta_master_transacciones,mtarjeta_master_consumo,cprestamos_personales,mprestamos_personales,cprestamos_prendarios,mprestamos_prendarios,cprestamos_hipotecarios,…,Visa_mfinanciacion_limite_diff_prev2,Visa_Fvencimiento_diff_prev,Visa_Fvencimiento_diff_prev2,Visa_msaldototal_diff_prev,Visa_msaldototal_diff_prev2,Visa_msaldopesos_diff_prev,Visa_msaldopesos_diff_prev2,Visa_msaldodolares_diff_prev,Visa_msaldodolares_diff_prev2,Visa_mconsumospesos_diff_prev,Visa_mconsumospesos_diff_prev2,Visa_mconsumosdolares_diff_prev,Visa_mconsumosdolares_diff_prev2,Visa_mlimitecompra_diff_prev,Visa_mlimitecompra_diff_prev2,Visa_madelantopesos_diff_prev,Visa_madelantopesos_diff_prev2,Visa_madelantodolares_diff_prev,Visa_madelantodolares_diff_prev2,Visa_fultimo_cierre_diff_prev,Visa_fultimo_cierre_diff_prev2,Visa_mpagado_diff_prev,Visa_mpagado_diff_prev2,Visa_mpagospesos_diff_prev,Visa_mpagospesos_diff_prev2,Visa_mpagosdolares_diff_prev,Visa_mpagosdolares_diff_prev2,Visa_fechaalta_diff_prev,Visa_fechaalta_diff_prev2,Visa_mconsumototal_diff_prev,Visa_mconsumototal_diff_prev2,Visa_cconsumos_diff_prev,Visa_cconsumos_diff_prev2,Visa_cadelantosefectivo_diff_prev,Visa_cadelantosefectivo_diff_prev2,Visa_mpagominimo_diff_prev,Visa_mpagominimo_diff_prev2
i64,i64,i64,i64,i64,i64,i64,f64,f64,f64,f64,f64,i64,i64,i64,f64,f64,i64,f64,f64,f64,i64,f64,i64,i64,f64,i64,i64,f64,i64,i64,f64,i64,f64,i64,f64,i64,…,f64,i64,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,i64,i64,f64,f64,f64,f64,f64,f64,i64,i64,f64,f64,i64,i64,i64,i64,f64,f64
249320580,202101,1,0,0,63,279,508.8,17140.27,896.11,0.0,-330.19,8,1,1,0.0,0.0,4,43297.51,0.0,100.35,1,77358.39,2,9,27877.92,1,0,0.0,1,0,0.0,0,0.0,0,0.0,0,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
249320580,202102,1,0,0,63,280,2494.79,18472.46,2015.94,0.66,407.57,8,1,1,0.0,-31.07,4,34954.32,0.0,103.25,1,80323.91,2,9,30659.34,1,0,0.0,1,0,0.0,0,0.0,0,0.0,0,…,null,28,null,0.0,null,0.0,null,0.0,null,null,null,null,null,0.0,null,null,null,null,null,-7,null,0.0,null,null,null,null,null,28,null,null,null,null,null,null,null,0.0,null
249320580,202103,1,0,0,63,281,3207.66,19030.88,2519.75,0.0,586.45,8,1,1,0.0,0.0,4,44960.66,0.0,105.74,1,69725.36,2,13,41845.43,1,0,0.0,1,0,0.0,0,0.0,0,0.0,0,…,0.0,31,59,0.0,0.0,0.0,0.0,0.0,0.0,null,null,null,null,0.0,0.0,null,null,null,null,4,-3,0.0,0.0,null,null,null,null,31,59,null,null,null,null,null,null,0.0,0.0
249320580,202104,1,0,0,64,282,3122.15,19147.05,2472.63,0.0,553.72,8,1,1,0.0,0.0,4,43728.23,0.0,107.54,1,91103.07,2,11,34122.88,1,0,0.0,1,0,0.0,0,0.0,0,0.0,0,…,0.0,30,61,0.0,0.0,0.0,0.0,0.0,0.0,null,null,null,null,0.0,0.0,null,null,null,null,1,5,0.0,0.0,null,null,null,null,30,61,null,null,null,null,null,null,0.0,0.0
249320580,202105,1,0,0,64,283,3455.86,20443.48,2520.36,0.0,797.53,8,1,1,0.0,0.0,4,60910.65,0.0,108.84,1,95203.09,2,11,45436.8,1,0,0.0,1,0,0.0,0,0.0,0,0.0,0,…,0.0,31,61,0.0,0.0,0.0,0.0,0.0,0.0,null,null,null,null,0.0,0.0,null,null,null,null,3,4,0.0,0.0,null,null,null,null,31,61,null,null,null,null,null,null,0.0,0.0


##SLOPES


In [15]:
import duckdb
import pandas as pd

# 1️⃣ Conectar a DuckDB en disco
conn = duckdb.connect(database="mi_proyecto.duckdb")

# 2️⃣ Ajustar parámetros de memoria
conn.execute("SET memory_limit='4GB'")
conn.execute("SET threads=2")
conn.execute("SET preserve_insertion_order=false")

# 3️⃣ Registrar CSV como tabla si no lo hiciste antes
conn.execute("""
    CREATE OR REPLACE TABLE tabla_confe AS
    SELECT * FROM read_csv_auto('data/df_confe.csv', header=True)
""")

# 4️⃣ Obtener esquema y filtrar columnas numéricas
schema_df = conn.execute("DESCRIBE tabla_confe").fetchdf()

cols_numericas = schema_df.loc[
    (~schema_df['column_type'].isin(['INTEGER', 'VARCHAR'])) &
    (~schema_df['column_name'].str.endswith('_lag1')) &
    (~schema_df['column_name'].str.endswith('_avg_3')) &
    (~schema_df['column_name'].str.contains('diff')) &
    (~schema_df['column_name'].str.contains('min')) &
    (~schema_df['column_name'].str.contains('max')) &
    (~schema_df['column_name'].str.startswith('Visa_f')) &
    (~schema_df['column_name'].str.startswith('Master_f')),
    'column_name'
].tolist()

# Excluir columnas adicionales fijas
excluir = ['numero_de_cliente', 'foto_mes', 'internet', 'cliente_edad', 'cliente_antiguedad']
cols_numericas = [c for c in cols_numericas if c not in excluir]

# 5️⃣ Procesar columnas en bloques
block_size = 50  # ajusta según tu RAM
for i in range(0, len(cols_numericas), block_size):
    block_cols = cols_numericas[i:i+block_size]

    # Crear expresiones REGR_SLOPE para este bloque
    regr_expr = [
        f"""CAST(
            CASE
                WHEN NOT isfinite(REGR_SLOPE({col}, cliente_antiguedad)) THEN 0
                ELSE REGR_SLOPE({col}, cliente_antiguedad)
            END AS DOUBLE
        ) AS slope_{col}"""
        for col in block_cols
    ]

    # Crear tabla temporal de slopes de este bloque
    query_slopes = f"""
    CREATE OR REPLACE TABLE df_slopes_block AS
    SELECT numero_de_cliente,
           {', '.join(regr_expr)}
    FROM tabla_confe
    GROUP BY numero_de_cliente
    """
    conn.execute(query_slopes)

    # Si es el primer bloque, creamos la tabla final
    if i == 0:
        conn.execute("CREATE OR REPLACE TABLE tabla_confe_slopes AS SELECT * FROM tabla_confe")
    
    # Hacemos LEFT JOIN incremental
    conn.execute(f"""
    CREATE OR REPLACE TABLE tabla_confe_slopes AS
    SELECT t.*, s.*
    FROM tabla_confe_slopes t
    LEFT JOIN df_slopes_block s USING (numero_de_cliente)
    """)

# 6️⃣ Traer una muestra a Pandas
df_final_slopes = conn.execute("SELECT * FROM tabla_confe_slopes LIMIT 1000").fetchdf()
print(df_final_slopes.head())


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

   numero_de_cliente  foto_mes  active_quarter  cliente_vip  internet  \
0          745470821    202103               1            0         0   
1          745470821    202104               1            0         0   
2          745470821    202105               1            0         0   
3          745470821    202106               1            0         0   
4          745479642    202101               1            0         0   

   cliente_edad  cliente_antiguedad  mrentabilidad  mrentabilidad_annual  \
0            54                 123       -2549.12             -34446.58   
1            54                 124         -55.40             -38196.66   
2            54                 125       -1471.18             -41131.71   
3            54                 126       -2161.27             -42742.65   
4            45                  60         -82.53              -2801.55   

   mcomisiones  ...  slope_Visa_mconsumosdolares  slope_Visa_mlimitecompra  \
0      5238.89  ...       

In [16]:
# Exportar todo a un nuevo CSV
conn.execute("""
COPY tabla_confe_slopes 
TO 'data/df_confe_slopes.csv' 
WITH (HEADER, DELIMITER ',')
""")


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [17]:
df_final_slopes.head(10)


,numero_de_cliente,foto_mes,active_quarter,cliente_vip,internet,cliente_edad,cliente_antiguedad,mrentabilidad,mrentabilidad_annual,mcomisiones,...,slope_Visa_mconsumosdolares,slope_Visa_mlimitecompra,slope_Visa_madelantopesos,slope_Visa_madelantodolares,slope_Visa_mpagado,slope_Visa_mpagospesos,slope_Visa_mpagosdolares,slope_Visa_mconsumototal,slope_Visa_cconsumos,slope_Visa_cadelantosefectivo
0,745470821,202103,1,0,0,54,123,-2549.12,-34446.58,5238.89,...,1.014857,16801.972571,0.0,0.0,0.0,10961.798000,0.0,4087.226857,0.971429,0.0
1,745470821,202104,1,0,0,54,124,-55.40,-38196.66,6206.08,...,1.014857,16801.972571,0.0,0.0,0.0,10961.798000,0.0,4087.226857,0.971429,0.0
2,745470821,202105,1,0,0,54,125,-1471.18,-41131.71,7082.49,...,1.014857,16801.972571,0.0,0.0,0.0,10961.798000,0.0,4087.226857,0.971429,0.0
3,745470821,202106,1,0,0,54,126,-2161.27,-42742.65,2827.15,...,1.014857,16801.972571,0.0,0.0,0.0,10961.798000,0.0,4087.226857,0.971429,0.0
4,745479642,202101,1,0,0,45,60,-82.53,-2801.55,-68.18,...,0.000000,0.000000,0.0,0.0,0.0,808.007143,0.0,77.905429,0.000000,0.0
5,745479642,202102,1,0,0,46,61,405.20,-2882.69,105.85,...,0.000000,0.000000,0.0,0.0,0.0,808.007143,0.0,77.905429,0.000000,0.0
6,745479642,202103,1,0,0,46,62,363.67,-3276.20,111.58,...,0.000000,0.000000,0.0,0.0,0.0,808.007143,0.0,77.905429,0.000000,0.0
7,745479642,202104,1,0,0,46,63,450.25,-3730.73,195.60,...,0.000000,0.000000,0.0,0.0,0.0,808.007143,0.0,77.905429,0.000000,0.0
8,745479642,202105,1,0,0,46,64,331.90,-1494.44,99.10,...,0.000000,0.000000,0.0,0.0,0.0,808.007143,0.0,77.905429,0.000000,0.0
9,745479642,202106,1,0,0,46,65,98.42,333.30,101.83,...,0.000000,0.000000,0.0,0.0,0.0,808.007143,0.0,77.905429,0.000000,0.0
